In [2]:
import os
from dotenv import load_dotenv

api_key = os.environ['GOOGLE_API_KEY']

In [3]:
from langchain.llms import GooglePalm
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import FewShotPromptTemplate
from langchain import PromptTemplate

In [4]:
llm= GooglePalm(api_key = api_key, temperature=0.1)
llm("Give me 10 best movies according to IMDB rating")

"| Rank | Movie Title | IMDB Rating |\n|---|---|---|\n| 1 | The Shawshank Redemption | 9.3 |\n| 2 | The Godfather | 9.2 |\n| 3 | The Dark Knight | 9.0 |\n| 4 | 12 Angry Men | 8.9 |\n| 5 | The Godfather Part II | 8.9 |\n| 6 | The Lord of the Rings: The Return of the King | 8.9 |\n| 7 | Pulp Fiction | 8.9 |\n| 8 | Schindler's List | 8.9 |\n| 9 | The Good, the Bad and the Ugly | 8.8 |\n| 10 | The Silence of the Lambs | 8.6 |"

In [5]:
loader = CSVLoader(file_path= 'faqs_data.csv', source_column='query')
data = loader.load()

data[:2]

[Document(page_content='query: What is the purpose of online education?\nresponse: Online education provides flexibility for learners to access courses and materials from anywhere, at any time, fostering a more accessible learning environment.', metadata={'source': 'What is the purpose of online education?', 'row': 0}),
 Document(page_content='query: How do I enroll in a course?\nresponse: To enroll in a course, visit our website, select the desired course, and follow the on-screen instructions for registration and payment.', metadata={'source': 'How do I enroll in a course?', 'row': 1})]

In [6]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
# It gives embedding based on instruct parameter
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

c:\Users\Kishan\anaconda3\envs\mlenv2\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
e[:5]

[-0.043898068368434906,
 0.007685542572289705,
 -0.009231895208358765,
 0.024496261030435562,
 0.03359226509928703]

In [10]:
vectordb = FAISS.from_documents(documents= data, embedding= instructor_embeddings)

In [15]:
#Retreiver takes the input sentence and finds the relevant docs

retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("Is there a way to contact the support team?")
rdocs

[Document(page_content='query: How can I contact customer support?\nresponse: You can reach our customer support team via email at support@example.com or through the live chat feature on our website during business hours.', metadata={'source': 'How can I contact customer support?', 'row': 6}),
 Document(page_content='query: Is there a trial period for courses?\nresponse: Some courses offer a trial period. Check the course page for information on trial options, or contact support for assistance.', metadata={'source': 'Is there a trial period for courses?', 'row': 12}),
 Document(page_content='query: How can I reset my password?\nresponse: Visit the login page, click on "Forgot Password," and follow the instructions sent to your registered email to reset your password securely.', metadata={'source': 'How can I reset my password?', 'row': 4}),
 Document(page_content='query: How do I enroll in a course?\nresponse: To enroll in a course, visit our website, select the desired course, and fol

In [24]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [25]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [23]:
#It finds the multiple questions from the data and tries to give answers merging them logically
chain("How can I enroll in the course and is there any financial aid policy?")

{'query': 'How can I enroll in the course and is there any financial aid policy?',
 'result': 'To enroll in a course, visit our website, select the desired course, and follow the on-screen instructions for registration and payment. Yes, we offer financial aid for eligible students. Check the course details or contact our financial aid department for information on the application process.',
 'source_documents': [Document(page_content='query: Are there financial aid options?\nresponse: Yes, we offer financial aid for eligible students. Check the course details or contact our financial aid department for information on the application process.', metadata={'source': 'Are there financial aid options?', 'row': 5}),
  Document(page_content='query: How do I enroll in a course?\nresponse: To enroll in a course, visit our website, select the desired course, and follow the on-screen instructions for registration and payment.', metadata={'source': 'How do I enroll in a course?', 'row': 1}),
  Doc

In [26]:
# It will throw 'I don't know' incase the csv doesn't have any info about the question
chain("Do you have any plan to launch GenAI course?")

{'query': 'Do you have any plan to launch GenAI course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='query: Are there courses on Deep Learning and Neural Nets?\nresponse: Yes, we offer specialized courses on Deep Learning, covering topics such as neural networks, convolutional neural networks (CNNs), and recurrent neural networks (RNNs).', metadata={'source': 'Are there courses on Deep Learning and Neural Nets?', 'row': 27}),
  Document(page_content='query: Can I get mentorship during the ML projects?\nresponse: Yes, some courses offer mentorship options where experienced professionals in the field provide guidance and feedback on your ML projects. Check the course details for mentorship availability.', metadata={'source': 'Can I get mentorship during the ML projects?', 'row': 29}),
  Document(page_content='query: How often are ML courses updated with new content?\nresponse: We regularly update our courses to reflect the latest advancements in the field. E